In [ ]:
# install ktrain
!pip3 install ktrain

In [ ]:
# import numpy
import numpy as np 
# import pandas
import pandas as pd
# import tensorflow
import tensorflow as tf
# import ktrain
import ktrain
from ktrain import text

**STEP 0: Download IMDb movie review dataset** 

In [26]:
dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

In [19]:
# set path to dataset
import os.path
#dataset = '/root/.keras/datasets/aclImdb'
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), 'aclImdb')
print(IMDB_DATADIR)

/root/.keras/datasets/aclImdb


**STEP 1: Load and Preprocess the Dataset**

In [20]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder(IMDB_DATADIR, 
                                                                       maxlen=500, 
                                                                       preprocess_mode='bert',
                                                                       train_test_names=['train', 
                                                                                         'test'],
                                                                       classes=['pos', 'neg'])

detected encoding: utf-8
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [21]:
x_train[0].shape

(25000, 500)

**STEP 2: Load a pretrained BERT model and wrap it in a ktrain.Learner object**

In [27]:
model = text.text_classifier(name ='bert' ,
                             train_data= (x_train , y_train),
                             preproc =preproc)

Is Multi-Label? False
maxlen is 500
done.


In [28]:
# Get lerning rate 
learner = ktrain.get_learner(model = model,
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=6)

**STEP 3: Train and Fine-Tune the Model on the IMDb dataset**


In [30]:
# optimal learning rate 
# Note this take days to run 
#learner.lr_find()
#learner.lr_plot()

In [31]:
#optimal learning rate for this model is 2e-5
learner.fit_onecycle(lr =2e-5,epochs= 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 25000 samples, validate on 25000 samples
25000/25000 [==============================] - 3458s 138ms/sample - loss: 0.2526 - accuracy: 0.8937 - val_loss: 0.1639 - val_accuracy: 0.9377


In [32]:
learner

**STEP 4: Prediction**


In [33]:
predictor = ktrain.get_predictor(learner.model ,preproc)

In [34]:
data = ['this movie was horrible . the plote was really boring . Acting was okay  though' , 
        'the film really sucked. i want my money back ',
        'what a beautifull movie. great plot ,great acting. will see it again',
        'the film really sucked . there is no plot and actingwas bad .']

In [35]:
predictor.predict(data)

['neg', 'neg', 'pos', 'neg']

**STEP 5: Save model**

In [36]:
predictor.save('/content/bert')